In [38]:
import os, json, re
import zipfile
from tqdm import tqdm
import numpy as np
from keras.layers import LSTM, Dense, Input, Embedding
from keras.models import Model
import tensorflow as tf

In [3]:
main_direc = "데이터셋/147.속성기반 감정분석 데이터/01-1.정식개방데이터/Training/"
second_direc = "데이터셋/147.속성기반 감정분석 데이터/01-1.정식개방데이터/Validation/"
data_label = "02.라벨링데이터/"

In [4]:
# label_data = []
# label_data2 = []
# label_data += [main_direc + data_label + i for i in os.listdir(main_direc + data_label)]
# label_data2 += [second_direc + data_label + i for i in os.listdir(second_direc + data_label)]

In [5]:
# for d in label_data:
#     data = zipfile.ZipFile(d)
#     data.extractall(main_direc + data_label)

# for d in label_data2:
#     data = zipfile.ZipFile(d)
#     data.extractall(second_direc + data_label)

In [6]:
label_data = []
test_label_data = []

label_data += [main_direc + data_label + i for i in os.listdir(main_direc + data_label)]
test_label_data += [second_direc + data_label + i for i in os.listdir(second_direc + data_label)]


print(len(label_data), len(test_label_data))

2004 256


In [7]:
out_text = []
out_polarity =[]
test_text = []
test_polarity = []

for link in tqdm(label_data):
    with open(link, "r", encoding="utf-8") as f:
        data3 = json.load(f)
    for i in range(len(data3)):
        if 'GeneralPolarity' in data3[i].keys():
            x = data3[i]['GeneralPolarity']
            x = int(x) + 1
            x2 = np.zeros(3, dtype = int)
            x2[x] = 1
            out_polarity.append(x2)  # 문장 전체 긍정 유무
            x = data3[i]['RawText']
            x = ' '.join(re.compile("[가-힣|ㄱ-ㅎ ]+").findall(x))
            out_text.append(x)  # 문장 전체

    
for link in tqdm(test_label_data):
    with open(link, "r", encoding="utf-8") as f:
        data3 = json.load(f)
    for i in range(len(data3)):
        if 'GeneralPolarity' in data3[i].keys():
            x = data3[i]['GeneralPolarity']
            x = int(x) + 1
            x2 = np.zeros(3, dtype = int)
            x2[x] = 1
            test_polarity.append(x2)  # 문장 전체 긍정 유무
            x = data3[i]['RawText']
            x = ' '.join(re.compile("[가-힣|ㄱ-ㅎ ]+").findall(x))
            test_text.append(x)  # 문장 전체

print(len(out_text),len(out_polarity),len(test_text),len(test_polarity))

100%|███████████████████████████████████████████████████████████████████████████████| 256/256 [00:00<00:00, 333.04it/s]

183432 183432 24869 24869


In [8]:
vocab = set()
for i in out_text:
    i = list(i)
    for j in i:
        vocab.add(j)
        
for i in test_text:
    i = list(i)
    for j in i:
        vocab.add(j)    

In [9]:
vocab = sorted(list(vocab))

In [18]:
# max_length = max([max([len(i) for i in out_text]), max([len(i) for i in test_text])])
max_length = 200

In [19]:
idx2char = {i : vocab[i] for i in range(len(vocab))}
char2idx = {vocab[i] : i for i in range(len(vocab))}

In [20]:
def padding_and_preprocessing(data, max_length):
    out = np.zeros(max_length, dtype = int)
    for i, d in enumerate(data):
        if i+1 > max_length:
            break
        out[i] = char2idx[d]
    return out

input_datas = []
for o_t in out_text:
    input_datas.append(padding_and_preprocessing(o_t, max_length))
    
input_test_datas = []
for o_t in test_text:
    input_test_datas.append(padding_and_preprocessing(o_t, max_length))

In [21]:
input_datas = np.array(input_datas)
input_test_datas = np.array(input_test_datas)
out_polarity = np.array(out_polarity)
test_polarity = np.array(test_polarity)

In [22]:
print(len(input_datas),len(out_polarity),len(input_test_datas),len(test_polarity))

183432 183432 24869 24869


In [23]:
inputs = Input(shape = (input_datas.shape[1],))
embed = Embedding(input_dim = len(char2idx) + 1, output_dim = 2**7)(inputs)
lstm1 = LSTM(2**6, return_sequences=True)(embed)
lstm2 = LSTM(2**5, return_sequences=True)(lstm1)
lstm3 = LSTM(2**4, return_sequences=False)(lstm2)
out = Dense(3, activation = "softmax")(lstm3)

In [24]:
model = Model(inputs = inputs, outputs = out)

In [25]:
model.compile(loss = "categorical_crossentropy", optimizer = "Adam")

In [42]:
model.load_weights("감정기반데이터.weights.h5")

In [26]:
model.fit(input_datas, out_polarity, batch_size = 2 ** 12, epochs = 1000, shuffle=True, validation_data = (input_test_datas, test_polarity))

Epoch 1/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 196s 4s/step - loss: 0.9117 - val_loss: 0.8771
Epoch 2/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 184s 4s/step - loss: 0.8807 - val_loss: 0.8738
Epoch 3/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 185s 4s/step - loss: 0.8794 - val_loss: 0.8730
Epoch 4/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 185s 4s/step - loss: 0.8801 - val_loss: 0.8695
Epoch 5/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 187s 4s/step - loss: 0.8744 - val_loss: 0.7482
Epoch 6/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - loss: 0.6679 - val_loss: 0.6003
Epoch 7/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 197s 4s/step - loss: 0.5668 - val_loss: 0.5373
Epoch 8/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 197s 4s/step - loss: 0.5199 - val_loss: 0.5061
Epoch 9/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 195s 4s/step - loss: 0.4887 - val_loss: 0.4772
Epoch 10/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 210s 5s/step - loss: 0.4568 - val_loss: 0.4607
Epoch 11/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 195s 4s/step - loss: 0.4229 - val_loss: 0.4210
Epoch 12/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 

KeyboardInterrupt: 

In [44]:
model.save_weights("감정기반데이터.weights.h5")

In [60]:
test_input = ""
test_input = ' '.join(re.compile("[가-힣|ㄱ-ㅎ ]+").findall(test_input))
test_input = padding_and_preprocessing(test_input, max_length)

out = model.predict(np.array([test_input]))
print(out)
out2 = np.argmax(out)
print(out2)

if out2 == 0:
    print("이 글은 {:.4f} % 확률로 부정적인 문장입니다.".format(out[0,0]))

elif out2 == 1:
    print("이 글은 {:.4f} % 확률로 중립적인 문장입니다.".format(out[0,1]))

elif out2 == 2:
    print("이 글은 {:.4f} % 확률로 긍정적인 문장입니다.".format(out[0,2]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[[0.02393688 0.36169118 0.61437196]]
2
이 글은 0.6144 % 확률로 긍정적인 문장입니다.


In [ ]:
[max_length, 

In [ ]:
with open("감정기반데이터.json", "w") with f:
    json.dump( ,f)